In [ ]:
%pip install torch torchvision norse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for norse: filename=norse-1.1.0-py3-none-any.whl size=1539018 sha256=9fad2ecb8f0fa0670bf30b5fd3826ae3bb56027b94ae516e6e8e040c9b6690a7
  Stored in directory: /root/.cache/pip/wheels/16/fc/0d/4cbb14992b7e5bb35482df57e887a2ab55cad9ea890501cf61
Successfully built norse


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import norse

class SNNNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = norse.torch.LConv2d(
            in_channels=1,
            out_channels=20,
            kernel_size=5,
            stride=1
        )
        self.conv2 = norse.torch.LConv2d(
            in_channels=20,
            out_channels=50,
            kernel_size=5,
            stride=1
        )
        self.fc1 = norse.torch.LILinearCell(4*4*50, 500)
        self.fc2 = norse.torch.LILinearCell(500, 10)
        self.max_pool = torch.nn.MaxPool2d(2)

    def forward(self, x):
        # Add time dimension
        batch_size = x.shape[0]
        time_steps = 50
        x = x.unsqueeze(0).repeat(time_steps, 1, 1, 1, 1)

        # Encode input as spikes using threshold
        x = (x > 0.5).float()

        # Conv layers
        # Apply convolutions to the entire 5D tensor at once
        spikes = self.conv1(x)
        # Apply max pooling along spatial dimensions for each time step and batch
        spikes = spikes.permute(1, 0, 2, 3, 4) # Permute to (batch, time, channels, height, width)
        spikes = spikes.reshape(batch_size * time_steps, spikes.shape[2], spikes.shape[3], spikes.shape[4]) # Reshape to (batch*time, channels, height, width)
        spikes = self.max_pool(spikes)
        spikes = spikes.reshape(batch_size, time_steps, spikes.shape[1], spikes.shape[2], spikes.shape[3]) # Reshape back to (batch, time, channels, height, width)
        spikes = spikes.permute(1, 0, 2, 3, 4) # Permute back to (time, batch, channels, height, width)

        spikes = self.conv2(spikes)
        # Apply max pooling along spatial dimensions for each time step and batch
        spikes = spikes.permute(1, 0, 2, 3, 4) # Permute to (batch, time, channels, height, width)
        spikes = spikes.reshape(batch_size * time_steps, spikes.shape[2], spikes.shape[3], spikes.shape[4]) # Reshape to (batch*time, channels, height, width)
        spikes = self.max_pool(spikes)
        spikes = spikes.reshape(batch_size, time_steps, spikes.shape[1], spikes.shape[2], spikes.shape[3]) # Reshape back to (batch, time, channels, height, width)
        spikes = spikes.permute(1, 0, 2, 3, 4) # Permute back to (time, batch, channels, height, width)

        # Flatten
        spikes = spikes.view(time_steps, batch_size, 4*4*50)

        # Fully connected layers
        # Iterate over the batch dimension
        outputs = []
        for i in range(batch_size):
            # Process each sample individually
            out, state = self.fc1(spikes[:, i, :])  # Pass one sample at a time
            out, state = self.fc2(out)
            outputs.append(out)  # Collect the outputs

        # Stack the outputs back into a single tensor
        spikes = torch.stack(outputs, dim=1)

        # Average over time steps
        return spikes.mean(0)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    losses = []

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = torch.nn.functional.cross_entropy(output, target)

        loss.backward()
        optimizer.step()
        losses.append(loss.item())

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    return torch.tensor(losses).mean()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            test_loss += torch.nn.functional.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')

    return test_loss, accuracy

In [ ]:
# Training settings
batch_size = 64
epochs = 10
learning_rate = 1e-3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(
    '../data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    '../data',
    train=False,
    transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False
)

# Initialize model and optimizer
model = SNNNetwork().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, optimizer, epoch)
    test_loss, accuracy = test(model, device, test_loader)

    print(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302570
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.277333
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.303344
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.371551
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.250102
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.177385
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.191238
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.108413
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.117726
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.081127

Test set: Average loss: 0.1072, Accuracy: 9657/10000 (96.57%)

Epoch: 1, Train Loss: 0.2984, Test Loss: 0.1072, Test Accuracy: 96.57%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.116721
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.067568
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.132048
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.290876
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.018686
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.155894
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.060432
Train Epoch: 2 [44800/60000